In [1]:
import numpy as np
import torch.nn.functional as nnFunctions
from torch.autograd import Variable
import torch
from torch import nn
import torchvision
from PIL import Image
import skimage.io as io
import pickle

In [2]:
class myConvNet(nn.Module):
    #constructor
    def __init__(self):
        super(myConvNet, self).__init__()
        #defining layers in convnet
        #input size=1*657*1625
        self.conv1 = nn.Conv2d(512,512, kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(512,256, kernel_size=3,stride=1,padding=1)
        #self.bn1=nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1)
        self.pconv1= nn.Conv2d(256,128, kernel_size=(3,3),stride=1,padding=(1,1))
        #self.bn2=nn.BatchNorm2d(64)
        self.pconv2= nn.Conv2d(256,128, kernel_size=(3,7),stride=1,padding=(1,3))
        self.pconv3= nn.Conv2d(256,128, kernel_size=(7,3),stride=1,padding=(3,1))
        
        self.conv4= nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1)
        self.conv5= nn.Conv2d(64,1,kernel_size=3,stride=1,padding=1)
        
    def forward(self, x,index1,index2,index3,index4):
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv1(x)),indices=index4,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.relu(self.conv2(x))
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv3(x)),indices=index3,kernel_size=(2,2),stride=(2,2))
        #parallel conv
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.pconv1(x)+self.pconv2(x)+self.pconv3(x)),indices=index2,kernel_size=(2,2),stride=(2,2))
        
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv4(x)),indices=index1,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.relu(self.conv5(x))
        return x

In [3]:
vgg16=torchvision.models.vgg16(pretrained=True)

In [4]:
modified_pretrained = nn.Sequential(*list(vgg16.features.children())[:-1])

In [5]:
modified_pretrainedified_pretrained

Sequential (
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU (inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU (inplace)
  (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU (inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU (inplace)
  (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU (inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU (inplace)
  (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (18): ReLU (inplace)
  (19): Conv2d(512,

In [6]:
class finalConvNet(nn.Module):
    global index1,index2,index3,index4
    #constructor
    def __init__(self,mynet,vgg16):
        super(finalConvNet, self).__init__()
        self.vgg=vgg16
        self.custom_net=mynet
        
        for i in range(30):
            if(i<14):
                self.vgg[i].requires_grad=False
            else:
                self.vgg[i].requires_grad=True
            if(i==4):
                self.vgg[i].return_indices=True
            elif(i==9):
                self.vgg[i].return_indices=True
            elif(i==16):
                self.vgg[i].return_indices=True
            elif(i==23):
                self.vgg[i].return_indices=True
        
    def forward(self, x):
        #4 max pool layers
        for layers in range(30):
            if(layers==4):
                x,index1=self.vgg[layers](x)
            elif(layers==9):
                x,index2=self.vgg[layers](x)
            elif(layers==16):
                x,index3=self.vgg[layers](x)
            elif(layers==23):
                x,index4=self.vgg[layers](x)
            else:
                x=self.vgg[layers](x)
        x=self.custom_net(x,index1,index2,index3,index4)
        return x

In [7]:
net=torch.load('../net_transfer_learning_5_intermediate5.txt')

In [8]:
net.cuda()

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
np_data=np.fromfile('../data/testImagesNumpy_512x512_color.txt')

In [ ]:
np_data.shape

In [ ]:
np_data=np_data.reshape(141,512,512,-1)

In [ ]:
from numpy import newaxis

In [ ]:
np_data=np_data.astype(np.float32)
np_data=np.transpose(np_data,[0,3,1,2])

In [ ]:
np_data.shape

In [ ]:
import torch.utils.data as data_utils

In [ ]:
features=torch.from_numpy(np_data)
targets=torch.from_numpy(np.zeros(np_data.shape))

In [ ]:
display = data_utils.TensorDataset(features,targets)
display_loader = data_utils.DataLoader(display, batch_size=1, shuffle=False)

In [ ]:
dataiter = iter(display_loader)

In [ ]:
images, labels = dataiter.next()

predicted = net(Variable(images).cuda())

In [ ]:
predicted=predicted.data.cpu().numpy()

In [ ]:
predicted.shape

In [ ]:
for i in range(1):
    for j in range(1):
        for k in range(512):
            for l in range(512):
                if(predicted[i][j][k][l]>200):
                    predicted[i][j][k][l]=255
                else:
                    predicted[i][j][k][l]=0

In [ ]:
import matplotlib.pyplot as plt

In [9]:
%matplotlib inline
def imshow(img,batch,isMask=False,num=0):
    npimg = img.numpy()
    for j in range(batch):
        print (npimg.shape)
        
        if(isMask):
            finImg=np.transpose(npimg[j], (1,2,0)).reshape(512,512).astype(np.float32)
            io.imsave('../Transfer_results/train_'+str(num)+'.png',finImg.astype(np.int32))
            plt.imshow(finImg,cmap='gray')
        else:
            finImg=np.transpose(npimg[j], (1,2,0)).reshape(512,512,3).astype(np.float32)
            plt.imshow(finImg,cmap='gray')
        name='img_'+str(j+1)
#         if(isMask):
#             name=name+'mask'
#             tmp=Image.fromarray(finImg)
#             print(tmp.mode)
#         else: 
#             tmp=Image.fromarray(finImg, 'RGB')
#         name=name+'.jpeg'
#         tmp.save('BTP_result/'+name)
        plt.show()

In [ ]:
imshow(images,1)

In [ ]:
imshow(torch.from_numpy(predicted),1,True,17)